<a href="https://colab.research.google.com/github/MeganJard/RSHB_Project_Bogdanov_Jaroslav/blob/main/Data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymorphy2

In [2]:
import nltk
import re
import pymorphy2
import json

In [ ]:
nltk.download("stopwords") # стоп слова
nltk.download('wordnet') # проводит лемматизацию
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_ru')
nltk.download('punkt')
from nltk.corpus import stopwords

morph = pymorphy2.MorphAnalyzer()

lemmatize = nltk.WordNetLemmatizer()

In [76]:
towar_uslugi = {}
uslugi_towar = {}
all_uslugi = set()
with open('/content/materials_fully.json') as json_file:
    materials_data = json.load(json_file)
with open('/content/stroyka_fully.json') as json_file:
    stroyka_data = json.load(json_file)
with open('/content/remont_fully.json') as json_file:
    stroyka_data |= json.load(json_file)
with open(r'/content/Соответствие категорий товаров и услуг.csv', encoding='utf-8') as f:
    for line in f:
        line = line.replace('\n', '')
        towar = line.split(';')[0]
        uslugi = line.split(';')[1].split(', ')
        towar_uslugi[towar] = uslugi

for towar in towar_uslugi:
    for usluga in towar_uslugi[towar]:
        all_uslugi.add(usluga)
for usluga in list(all_uslugi):
    uslugi_towar[usluga] = []
    for towar in towar_uslugi:
        if usluga in towar_uslugi[towar]:
            uslugi_towar[usluga].append(towar)


In [8]:
custom_stopwords = open('/content/стопслова.txt').read().replace(' ', '').replace('\t', '').split('\n')


In [9]:
def get_tokens(input):
  input = input.lower()
  regular = r'[\*+\#+\№\"\-+\+\=+\?+\&\^\.+\;\,+\>+\(\)\/+\:\\+\–+]'
  regular_url = r'(http\S+)|(www\S+)|([\w\d]+www\S+)|([\w\d]+http\S+)'
  input = re.sub(regular, '', input)
  input = re.sub(regular_url, r'URL', input)
  input = re.sub(r'(\d+\s\d+)|(\d+)',' NUM ', input)
  input = re.sub(r'\s+', ' ', input)
  text = nltk.word_tokenize(input, language='russian')
  text = [lemmatize.lemmatize(word) for word in text]
  filtered_words = [word for word in text if morph.parse(word)[0].normal_form not in stopwords.words('russian')+custom_stopwords]
  tagged = nltk.pos_tag(filtered_words, lang='rus')
  for i in range(len(tagged)):
    if tagged[i][1] in ['NONLEX', 'INTJ', 'V', 'ADV']:
      tagged[i] = ''
  tagged = list(filter(('').__ne__, tagged))
  for i in range(len(tagged)):
    tagged[i] = morph.parse(tagged[i][0])[0].normal_form

  return tagged

In [ ]:
stroyka_dict = {}
materials_dict = {}
for item in stroyka_data:
  item_dict = {}
  item_dict['url'] = item.split('~')[2]
  item_dict['name'] = list(set(get_tokens(item.split('~')[0])))
  item_dict['opis'] = list(set(get_tokens(stroyka_data[item])))
  item_dict['real_name'] = item.split('~')[0]
  item_dict['categ'] = stroyka_data[item].split('\n')[0]
  stroyka_dict[item.split('~')[1]] = item_dict
for item in materials_data:
  item_dict = {}
  item_dict['url'] = item.split('~')[2]
  item_dict['name'] = list(set(get_tokens(item.split('~')[0])))
  item_dict['real_name'] = item.split('~')[0]
  item_dict['opis'] = list(set(get_tokens(materials_data[item])))
  item_dict['categ'] = materials_data[item].split('\n')[0]
  materials_dict[item.split('~')[1]] = item_dict

In [83]:
#теперь посчитаем подходящие материалы для каждой услуги
answer = dict()

for usluga in stroyka_dict:
  answer[usluga] = []
  for material in materials_dict:
    intersection_opis = set(stroyka_dict[usluga]['opis'])&set(materials_dict[material]['opis'])
    intersection_title = set(stroyka_dict[usluga]['opis'])&set(materials_dict[material]['name'])
    if len(materials_dict[material]['opis']) and len(stroyka_dict[usluga]['opis']):
        try:
          if not (materials_dict[material]['categ'] in uslugi_towar[stroyka_dict[usluga]['categ']]):
            continue
        except Exception as e:
          all_probs.add(materials_dict[material]['categ'])
          continue
        answer[usluga].append({material: len(intersection_opis)/((len(materials_dict[material]['opis'])+len(stroyka_dict[usluga]['opis'])))+2*len(intersection_title)/(len(stroyka_dict[usluga]['opis']))})
  answer[usluga] = sorted(answer[usluga], key=lambda x: list(x.values())[0], reverse=True)
  answer_with_words = dict()
  for title in answer:
    buffer = []
    for i in range(min(5, len(answer[title]))):
      buffer.append([materials_dict[list(answer[title][i].keys())[0]]['real_name'], materials_dict[list(answer[title][i].keys())[0]]['url']])
    answer_with_words[stroyka_dict[title]['real_name']+'~~'+stroyka_dict[title]['url']] = buffer

In [84]:
with open('/content/uslugi_to_materials.json', 'w', encoding='utf-8') as fp:
    json.dump(answer_with_words, fp, ensure_ascii=False)

In [79]:
#Теперь посчитаем подходящие услуги для каждого материала
answer = dict()
all_probs = set()
for material in materials_dict:
  answer[material] = []
  for usluga in stroyka_dict:
    intersection_opis = set(stroyka_dict[usluga]['opis'])&set(materials_dict[material]['opis'])
    intersection_title = set(stroyka_dict[usluga]['name'])&set(materials_dict[material]['opis'])
    try:
      if len(materials_dict[material]['opis']) and len(stroyka_dict[usluga]['opis']):
        try:

          if not (stroyka_dict[usluga]['categ'] in towar_uslugi[materials_dict[material]['categ']]):
            continue
        except Exception as e:
          all_probs.add(materials_dict[material]['categ'])
          continue
        answer[material].append({usluga: len(intersection_opis)/((len(materials_dict[material]['opis'])+len(stroyka_dict[usluga]['opis'])))+2*len(intersection_title)/(len(materials_dict[material]['opis']))})
    except Exception as e:
      print('ERROR', e)
  answer[material] = sorted(answer[material], key=lambda x: list(x.values())[0], reverse=True)
  answer_with_words = dict()
  for title in answer:
    buffer = []
    for i in range(min(5, len(answer[title]))):
      buffer.append([stroyka_dict[list(answer[title][i].keys())[0]]['real_name'], stroyka_dict[list(answer[title][i].keys())[0]]['url']])
    answer_with_words[materials_dict[title]['real_name']+'~~'+materials_dict[title]['url']] = buffer

In [81]:
with open('/content/materials_to_uslugi.json', 'w', encoding='utf-8') as fp:
    json.dump(answer_with_words, fp, ensure_ascii=False)

In [ ]:
#теперь можно проанализировать тэги, чтобы пополнить библиотеку стоп-слов
stop_words = dict()
all_tags_to_freq = dict()
for material in stroyka_dict:
  for tag in stroyka_dict[material]['name']:
    if tag in all_tags_to_freq:
      all_tags_to_freq[tag] += 1
    else:
      all_tags_to_freq[tag] = 1
  for tag in stroyka_dict[material]['opis']:
    if tag in all_tags_to_freq:
      all_tags_to_freq[tag] += 1
    else:
      all_tags_to_freq[tag] = 1

In [ ]:
all_tags = sorted(all_tags_to_freq, key=lambda x: all_tags_to_freq[x], reverse=True)
for i in all_tags:
  print(i, all_tags_to_freq[i])

In [ ]:
answer_with_words